### Importanto Bibliotecas

In [ ]:
import cv2 
import os
from PIL import Image
import numpy as np

### Inicializando variáveis

In [ ]:
root_dir = './data/no_label'
source_dir = os.path.join(root_dir, 'collected_images')
resized_dir = os.path.join(root_dir, 'resized_data')    
segmented_dir = os.path.join(root_dir, 'segmented_dir')   

width = []
height = []

### Calculando média das dimensões

In [ ]:
for file in os.listdir(source_dir):
    img_path = os.path.join(source_dir, file)

    image = Image.open(img_path)
    w, h = image.size

    width.append(w)
    height.append(h)

width_mean = int(sum(width) / len(width))
height_mean = int(sum(height) / len(height))

print(f"Média de largura: {int(width_mean)}")
print(f"Média de altura: {int(height_mean)}")



### Redimensionando o tamanho das imagens

In [ ]:
dim = (width_mean, height_mean)
dim
if not os.path.exists(resized_dir):
    os.makedirs(resized_dir)

for filename in os.listdir(source_dir):
    if filename.endswith('.jpg'):
        img = cv2.imread(os.path.join(source_dir, filename))
        img_resized = cv2.resize(img, dim, cv2.INTER_AREA)
        cv2.imwrite(os.path.join(resized_dir, filename), img_resized)
    else:
        ...

### Segmentação das Imagens

In [ ]:
if not os.path.exists(segmented_dir):
    os.makedirs(segmented_dir)
        
for filename in os.listdir(resized_dir):
    img = cv2.imread(os.path.join(resized_dir, filename))

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower = np.array([35, 10, 10])  # Verde mais escuro
    upper = np.array([80, 200, 200])# Verde mais claro

    mask = cv2.inRange(img_hsv, lower, upper)

    imageSegmented = cv2.bitwise_and(img, img, mask=mask)

    cv2.imwrite(os.path.join(segmented_dir, filename), imageSegmented)   